In [2]:
import pandas as pd
import numpy as np
from coredotfinance.data import KrxReader
krx = KrxReader()
import dart_fss as dart
from tqdm.auto import tqdm
from time import sleep
import FinanceDataReader as fdr
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [6]:
df_nasdaq = fdr.StockListing('NASDAQ')
df_nasdaq


100%|██████████| 4634/4634 [00:46<00:00, 98.75it/s] 


,Symbol,Name,Industry,IndustryCode
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",571060
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,572010
2,AMZN,Amazon.com Inc,다양한 소매업,534020
3,GOOGL,Alphabet Inc Class A,소프트웨어 및 IT서비스,572010
4,TSLA,Tesla Inc,자동차 및 자동차 부품,531010
...,...,...,...,...
4629,GBBKR,Global Blockchain Acquisition Rights,기타 금융업,556010
4630,ZTOPU,Zi Toprun Acquisition Units,기타 금융업,556010
4631,MOB,Mobilicom Ltd ADR,NaN,NaN
4632,RENE,Cartesian Growth Corporation II Class A,기타 금융업,556010


In [80]:
mc_nasdaq_df = []
for i in tqdm(range(len(df_nasdaq))):
    if (i + 1) % 10 == 0:
        sleep(1)
    
    name = df_nasdaq.loc[i, 'Name']
    symbol = df_nasdaq.loc[i, 'Symbol']
    industry = df_nasdaq.loc[i, 'Industry']
    if symbol == np.nan or industry == np.nan:
        continue
    
    try:
        series = fdr.DataReader(symbol, '20220701', '20220701')
        market_cap = series.iloc[0, 3] * series.iloc[0, 4]
        mc_nasdaq_df.append([name, industry, market_cap])
    except:
        print(f'{symbol} failed.')
        continue
    
df = pd.DataFrame(mc_nasdaq_df, columns=['Industry', 'Name', 'MarketCap'])
df.to_csv('./data/nasdaq_marketcap2.csv')

  0%|          | 0/4634 [00:00<?, ?it/s]

HTTP 429 error. Retry after 86400 seconds (1 day, 0:00:00)
BIDU failed.
HTTP 429 error. Retry after 86400 seconds (1 day, 0:00:00)
KDP failed.
HTTP 429 error. Retry after 86399 seconds (23:59:59)
AEP failed.
HTTP 429 error. Retry after 86399 seconds (23:59:59)
PANW failed.
HTTP 429 error. Retry after 86399 seconds (23:59:59)
MNST failed.
HTTP 429 error. Retry after 86398 seconds (23:59:58)
TEAM failed.
HTTP 429 error. Retry after 86398 seconds (23:59:58)
KLAC failed.
HTTP 429 error. Retry after 86398 seconds (23:59:58)
KHC failed.
HTTP 429 error. Retry after 86397 seconds (23:59:57)
SNPS failed.
HTTP 429 error. Retry after 86396 seconds (23:59:56)
FTNT failed.
HTTP 429 error. Retry after 86396 seconds (23:59:56)
MAR failed.
HTTP 429 error. Retry after 86395 seconds (23:59:55)
EXC failed.
HTTP 429 error. Retry after 86395 seconds (23:59:55)
VOD failed.
ORLY failed.
HTTP 429 error. Retry after 86394 seconds (23:59:54)
CDNS failed.
HTTP 429 error. Retry after 86394 seconds (23:59:54)
PAYX

In [30]:
nasdaq_count = defaultdict(int)
for i in range(len(df_nasdaq)):
    industry = df_nasdaq.loc[i, 'Industry']
    nasdaq_count[industry] += 1
df = pd.DataFrame.from_dict(dict(nasdaq_count), orient='index').rename(columns={0:'Count'}).sort_values(by=['Count'], ascending=False)[:10]

fig = go.Figure(data=go.Bar(
    x = df.index,
    y = df['Count'],
))
fig.update_layout(title='Nasdaq industry count')
fig.show()

In [70]:
df = pd.DataFrame(mc_nasdaq_df, columns=['Industry', 'MarketCap']).groupby('Industry').sum()
df = df.sort_values(by=['MarketCap'], ascending=False)[:10]

df2 = []
for index in df.index:
    df2.append([index, nasdaq_count[index]])
df2 = pd.DataFrame(df2, columns=['Industry', 'Count']).set_index('Industry')

fig = go.Figure(
    data=[
        go.Bar(name='MarketCap Sum', x = df.index, y = df['MarketCap'], yaxis='y', offsetgroup=1),
        go.Bar(name='Count', x = df2.index, y = df2['Count'], yaxis='y2', offsetgroup=2)
    ],
    layout={
        'yaxis': {'title': 'MarketCap Sum Axis'},
        'yaxis2': {'title': 'Count Axis', 'overlaying': 'y', 'side': 'right'},
    }
)
# fig.update_layout(title='MarketCap Sum')
fig.show()

top_industries = list(df.index)
print(top_industries)

['소프트웨어 및 IT서비스', '반도체 및 반도체 장비', '자동차 및 자동차 부품', '컴퓨터, 전화 및 가전제품', '다양한 소매업', '생명과학 및 메디컬 리서치', '통신 서비스', '헬스케어 장비 및 용품', '호텔 및 엔터테인먼트', '제약']


In [72]:
df = (
    pd.DataFrame(mc_nasdaq_df, columns=["Industry", "MarketCap"])
    .groupby("Industry")
    .mean()
)
df = df.sort_values(by=["MarketCap"], ascending=False)[:10]

fig = go.Figure(
    data=go.Bar(
        x=df.index,
        y=df["MarketCap"],
    )
)
fig.update_layout(title="MarketCap Mean")
fig.show()

top_industries.extend(list(df.index))
top_industries = list(set(top_industries))


['호텔 및 엔터테인먼트', '다양한 소매업', '생명과학 및 메디컬 리서치', '컴퓨터, 전화 및 가전제품', '제약', '운송업', '통신 서비스', '반도체 및 반도체 장비', '자동차 및 자동차 부품', '음료', '복합 기업', '소프트웨어 및 IT서비스', '헬스케어 장비 및 용품', '전기 유틸리티']


In [75]:
from yfinance.utils import get_json

fundamentals = []
for i in tqdm(range(len(df_nasdaq))):
    symbol = df_nasdaq.loc[i, "Symbol"]
    industry = df_nasdaq.loc[i, "Industry"]

    try:
        fundamental = get_json("https://finance.yahoo.com/quote/" + symbol)["financialData"]
        profitMargins = fundamental["profitMargins"]
        grossProfits = fundamental["grossProfits"]
        totalRevenue = fundamental["totalRevenue"]
    except:
        print(symbol, "failed.")
        continue

    fundamentals.append([industry, profitMargins, grossProfits, totalRevenue])

fund_df = pd.DataFrame(fundamentals, columns=[])
fund_df.to_csv("./data/nasdaq_fund.csv")


  0%|          | 0/4634 [00:00<?, ?it/s]

META failed.
EQIX failed.
ABNB failed.
PSNY failed.
PPD failed.
CREE failed.
PBCT failed.
MSPR failed.
AZTA failed.
OPCH failed.
PGY failed.
PFHC failed.
SRNG failed.
TLND failed.
ALVO failed.
SRNGU failed.
VRTU failed.
EIGI failed.
OMP failed.
DSGR failed.
IONR failed.
TOP failed.
ALTA failed.
CLCT failed.
TNT failed.
CMLT failed.
WIFI failed.
BOWX failed.
SYM failed.
BPYPM failed.
DCRC failed.
PACWP failed.
CMLTU failed.
MAAC failed.
HOL failed.
INAQU failed.
FCBC failed.
INTR failed.
BOWXU failed.
DCRCU failed.
MSON failed.
GIG failed.
PRE failed.
PPGH failed.
MDCA failed.
SGHT failed.
HEC failed.
MAACU failed.
SVACU failed.
ACGLN failed.
HCNE failed.
SIER failed.
TDACU failed.
PPGHU failed.
BGXX failed.
DGNS failed.
LACQU failed.
KSMT failed.
IVCA failed.
FIII failed.
EGIO failed.
LSAK failed.
GSUN failed.
SVOK failed.
TLC failed.
PLAO failed.
GXGX failed.
CRSA failed.
DCRB failed.
TMTS failed.
HUDI failed.
PFBI failed.
PBCTP failed.
IVCAU failed.
BOCH failed.
BTMD failed.
CRSAU fa

ValueError: 0 columns passed, passed data had 4 columns

In [79]:
df = pd.DataFrame(fundamentals, columns=['Industry', 'ProfitMargins', 'GrossProfits', 'TotalRevenue'])
df.to_csv('./data/nasdaq_profit.csv')